In [1]:
from imp import reload
import utils as ul
from mls import Model as MD
import fw
#import park as sc
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Data generation

In [2]:
## X: n*p  W: p*m  Y: n*m
np.random.seed(0)
X, Y, W = MD.generate(n = 16)
n = Y.shape[0]
m = Y.shape[1]
p = X.shape[1]

In [4]:
from sklearn.linear_model import MultiTaskLasso
MultiTaskLasso(alpha=.001).fit(X, Y).score(X, Y)

0.99998909921955026

In [83]:
points = ul.mat2point(X,Y)
dataRDD = sc.parallelize(points,8).mapPartitions(ul.point2mat)

In [84]:
dataRDD.first()

[array([[ 1.76405235,  0.40015721,  0.97873798,  2.2408932 ],
        [ 1.86755799, -0.97727788,  0.95008842, -0.15135721]]),
 array([[ 0.14377827, -0.579834  , -0.26093079,  0.26925737, -0.93048297],
        [ 0.12501844, -0.29203879, -0.65073588, -0.15719248, -0.09999714]])]

In [88]:
dataRDD = dataRDD.map(lambda z: \
                      dict([('X',z[0]),('Y',z[1]), \
                            ('n',z[1].shape[0]), \
                            ('m',z[1].shape[1]), \
                            ('p',z[0].shape[1])])).persist()
dataRDD

PythonRDD[46] at RDD at PythonRDD.scala:48

In [90]:
dataRDD.first()

{'X': array([[ 1.76405235,  0.40015721,  0.97873798,  2.2408932 ],
        [ 1.86755799, -0.97727788,  0.95008842, -0.15135721]]),
 'Y': array([[ 0.14377827, -0.579834  , -0.26093079,  0.26925737, -0.93048297],
        [ 0.12501844, -0.29203879, -0.65073588, -0.15719248, -0.09999714]]),
 'm': 5,
 'n': 2,
 'p': 4}

## Initialization

In [37]:
np.random.seed(0)

T = 30               # number of iterations of FW
U = np.zeros((T,p))  # left singular vector
V = np.zeros((T,m))  # right singular vector

loss = np.zeros(T)

In [58]:
rdd = dataRDD.map(lambda x: dict([('data',x), ('param',fw.Param((x.p, x.m)))]))
rdd

PythonRDD[34] at RDD at PythonRDD.scala:48

In [61]:
rdd.first()

{'data': <fw.Data at 0x7f6175a6bb00>, 'param': <fw.Param at 0x7f61759cf940>}

In [64]:
rdd.first()['param'].W

array([[ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.]])

## Subfunctions

In [66]:
rdd = fw.gradient(rdd, MD)
rdd

PythonRDD[42] at RDD at PythonRDD.scala:48

In [79]:
foo = dict()

In [80]:
foo['a'] = 1

In [82]:
foo['b'].c = 2

KeyError: 'b'

In [67]:
rdd.first().grad

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 37.0 failed 1 times, most recent failure: Lost task 0.0 in stage 37.0 (TID 79, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 172, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 167, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/spark/python/pyspark/rdd.py", line 1306, in takeUpToNumLeft
    yield next(iterator)
  File "/home/zheng/ownCloud/Project/dFW/spark3/fw.py", line 40, in _f
    grad = rdd.map(lambda row: row.grad).reduce(add)
AttributeError: 'dic

In [55]:
fw.centralize(rdd)

(array([ 1.        ,  0.25811172,  0.34515718,  0.17857333]),
 array([-0.20328427,  0.32771366,  1.        , -0.30466848,  0.82997534]))

In [57]:
foo['hello'] = 'world'

NameError: name 'foo' is not defined

In [56]:
fw.warmstart(rdd).first()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 28.0 failed 1 times, most recent failure: Lost task 0.0 in stage 28.0 (TID 63, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 172, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 167, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/spark/python/pyspark/rdd.py", line 1306, in takeUpToNumLeft
    yield next(iterator)
  File "/home/zheng/ownCloud/Project/dFW/spark3/fw.py", line 50, in _f
    row.eigvec.u, row.eigvec.v = svd(row.grad)
AttributeError: 'Row' obje

In [ ]:
fw.warmstart(gradRDD).values().collect()

In [ ]:
fw.avgmix(gradRDD, 1)

In [ ]:
fw.warmstart(gradRDD).map(lambda x: x[1][1]).collect()

In [ ]:
fw.warmstart(gradRDD).map(lambda x: x[1][1]).reduce(lambda x,y: x+y)

In [ ]:
fw.poweriter(gradRDD, 0, lambda t: 10)

In [ ]:
np.array([fw.loground(i) for i in range(101)])

In [ ]:
u, v = fw.centralize(gradRDD)
u, v

In [ ]:
u, v = fw.regularize(u, v, 1)
u, v

In [ ]:
paramRDD = fw.broadcast(paramRDD, u, v)
paramRDD.values().first().D

In [ ]:
np.outer(u,v)

In [ ]:
fw.loss(paramRDD, modelRDD)

In [ ]:
paramRDD, _ = fw.naivestep(paramRDD)
paramRDD.values().first().a

In [ ]:
paramRDD = fw.descent(paramRDD)
paramRDD.values().first().W

In [ ]:
fw.loss(paramRDD, modelRDD)

In [ ]:
paramRDD = fw.ascent(paramRDD)
paramRDD.values().first().W

In [ ]:
fw.loss(paramRDD, modelRDD)

In [ ]:
paramRDD, _ = fw.linearsearch(paramRDD, modelRDD, gradRDD)
paramRDD.values().first().a

In [ ]:
paramRDD = fw.descent(paramRDD)
paramRDD.values().first().W

In [ ]:
fw.loss(paramRDD, modelRDD)

In [ ]:
paramRDD = fw.ascent(paramRDD)
paramRDD.values().first().W

## Main function

In [ ]:
np.random.seed(0)
paramRDD = modelRDD.mapValues(lambda x: pb.Param((x.p, x.m)))
fw.iterate(modelRDD, paramRDD, fw.centralize, fw.naivestep, 1, 0)

In [ ]:
np.random.seed(0)
paramRDD = modelRDD.mapValues(lambda x: pb.Param((x.p, x.m)))
fw.iterate(modelRDD, paramRDD, fw.centralize, fw.linearsearch, 1, 0)

In [ ]:
np.random.seed(0)
paramRDD = modelRDD.mapValues(lambda x: pb.Param((x.p, x.m)))
fw.iterate(modelRDD, paramRDD, fw.avgmix, fw.linearsearch, 1, 0)

In [ ]:
np.random.seed(0)
paramRDD = modelRDD.mapValues(lambda x: pb.Param((x.p, x.m)))
fw.iterate(modelRDD, paramRDD, lambda x,t: fw.poweriter(x,t,lambda t: 1), fw.linearsearch, 1, 0)

In [ ]:
np.random.seed(0)
paramRDD = modelRDD.mapValues(lambda x: pb.Param((x.p, x.m)))
fw.iterate(modelRDD, paramRDD, lambda x,t: fw.poweriter(x,t,lambda t: 20), fw.linearsearch, 1, 0)

In [ ]:
np.random.seed(0)
paramRDD = modelRDD.mapValues(lambda x: pb.Param((x.p, x.m)))
fw.iterate(modelRDD, paramRDD, lambda x,t: fw.poweriter(x,t,fw.loground), fw.linearsearch, 1, 0)

In [ ]:
np.random.seed(0)
v0 = np.random.randn(m)
np.random.seed(0)
paramRDD = modelRDD.mapValues(lambda x: pb.Param((x.p, x.m)))
fw.iterate(modelRDD, paramRDD, lambda x,t: fw.poweriter(x,t,fw.loground,v0), fw.linearsearch, 1, 0)